In [1]:
# Import necessary libraries
import pandas as pd
import glob
from pandas_profiling import ProfileReport
import numpy as np
import json
import duckdb
import re
import functions_project

/var/folders/p3/prw8fq7d4t350r6vj8kcn_rh0000gn/T/ipykernel_26443/4224327402.py:4: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


# clean oscar data from https://www.kaggle.com/datasets/unanimad/the-oscar-award

In [2]:
oscars = pd.read_csv('data/the_oscar_award.csv')
oscars.isna().sum()

year_film          0
year_ceremony      0
ceremony           0
category           0
name               4
film             315
winner             0
dtype: int64

In [3]:
oscars = oscars.dropna(subset=[ 'winner', 'film'])
oscars.isna().sum()

year_film        0
year_ceremony    0
ceremony         0
category         0
name             0
film             0
winner           0
dtype: int64

In [4]:
oscars

,year_film,year_ceremony,ceremony,category,name,film,winner
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False
...,...,...,...,...,...,...,...
10755,2022,2023,95,WRITING (Original Screenplay),Written by Daniel Kwan & Daniel Scheinert,Everything Everywhere All at Once,True
10756,2022,2023,95,WRITING (Original Screenplay),Written by Steven Spielberg & Tony Kushner,The Fabelmans,False
10757,2022,2023,95,WRITING (Original Screenplay),Written by Todd Field,Tár,False
10758,2022,2023,95,WRITING (Original Screenplay),Written by Ruben Östlund,Triangle of Sadness,False


In [5]:
#oscars = oscars.drop_duplicates(subset=["film", "year_film"], keep="first") 


oscars['year + title'] = oscars["year_film"].astype(str) + oscars["film"]
oscars

oscars = oscars.drop_duplicates(subset=['year + title'] ,keep='first')
oscars

,year_film,year_ceremony,ceremony,category,name,film,winner,year + title
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False,1927The Noose
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True,1927The Last Command
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False,1927A Ship Comes In
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True,19277th Heaven
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False,1927Sadie Thompson
...,...,...,...,...,...,...,...,...
10736,2022,2023,95,SHORT FILM (Live Action),Alice Rohrwacher and Alfonso Cuarón,Le Pupille,False,2022Le Pupille
10737,2022,2023,95,SHORT FILM (Live Action),Eirik Tveiten and Gaute Lid Larssen,Night Ride,False,2022Night Ride
10738,2022,2023,95,SHORT FILM (Live Action),Cyrus Neshvad,The Red Suitcase,False,2022The Red Suitcase
10750,2022,2023,95,WRITING (Adapted Screenplay),Written by Rian Johnson,Glass Onion: A Knives Out Mystery,False,2022Glass Onion: A Knives Out Mystery


In [6]:
by_cat=oscars.groupby(by='category', as_index=False).agg({'film':'count'})
by_cat=by_cat.sort_values(by='film', ascending=False, ignore_index=True)
print(len(by_cat))
display(by_cat.head(10))

101


,category,film
0,DOCUMENTARY (Short Subject),378
1,DOCUMENTARY (Feature),343
2,FOREIGN LANGUAGE FILM,288
3,ACTOR IN A SUPPORTING ROLE,282
4,ACTOR IN A LEADING ROLE,232
5,ACTOR,222
6,SHORT FILM (Live Action),221
7,SHORT FILM (Animated),210
8,SHORT SUBJECT (Cartoon),169
9,ACTRESS IN A LEADING ROLE,167


In [7]:
#Rewrite all categories in lower-case
oscars['category']=[word.lower() for word in oscars['category']]

oscars

/var/folders/p3/prw8fq7d4t350r6vj8kcn_rh0000gn/T/ipykernel_26443/1394384046.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oscars['category']=[word.lower() for word in oscars['category']]


,year_film,year_ceremony,ceremony,category,name,film,winner,year + title
0,1927,1928,1,actor,Richard Barthelmess,The Noose,False,1927The Noose
1,1927,1928,1,actor,Emil Jannings,The Last Command,True,1927The Last Command
2,1927,1928,1,actress,Louise Dresser,A Ship Comes In,False,1927A Ship Comes In
3,1927,1928,1,actress,Janet Gaynor,7th Heaven,True,19277th Heaven
4,1927,1928,1,actress,Gloria Swanson,Sadie Thompson,False,1927Sadie Thompson
...,...,...,...,...,...,...,...,...
10736,2022,2023,95,short film (live action),Alice Rohrwacher and Alfonso Cuarón,Le Pupille,False,2022Le Pupille
10737,2022,2023,95,short film (live action),Eirik Tveiten and Gaute Lid Larssen,Night Ride,False,2022Night Ride
10738,2022,2023,95,short film (live action),Cyrus Neshvad,The Red Suitcase,False,2022The Red Suitcase
10750,2022,2023,95,writing (adapted screenplay),Written by Rian Johnson,Glass Onion: A Knives Out Mystery,False,2022Glass Onion: A Knives Out Mystery


In [8]:
#call function map_cleaned_strings to apply in the dataframe on the columns title, director, cast, country
functions_project.map_cleaned_strings(oscars, 'name')
functions_project.map_cleaned_strings(oscars, 'film')
functions_project.map_cleaned_strings(oscars, 'name')
functions_project.map_cleaned_strings(oscars, 'film')

oscars

/Users/samvandenelsaker/Desktop/Big Data/project-Big-Data/functions_project.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfname[columnname] = dfname[columnname].map(dict_old_new)
/Users/samvandenelsaker/Desktop/Big Data/project-Big-Data/functions_project.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfname[columnname] = dfname[columnname].map(dict_old_new)
/Users/samvandenelsaker/Desktop/Big Data/project-Big-Data/functions_project.py:26: SettingWithCopyWarning: 
A value is trying to be set on

,year_film,year_ceremony,ceremony,category,name,film,winner,year + title
0,1927,1928,1,actor,richard barthelmess,the noose,False,1927The Noose
1,1927,1928,1,actor,emil jannings,the last command,True,1927The Last Command
2,1927,1928,1,actress,louise dresser,a ship comes in,False,1927A Ship Comes In
3,1927,1928,1,actress,janet gaynor,7th heaven,True,19277th Heaven
4,1927,1928,1,actress,gloria swanson,sadie thompson,False,1927Sadie Thompson
...,...,...,...,...,...,...,...,...
10736,2022,2023,95,short film (live action),alice rohrwacher and alfonso cuaron,le pupille,False,2022Le Pupille
10737,2022,2023,95,short film (live action),eirik tveiten and gaute lid larssen,night ride,False,2022Night Ride
10738,2022,2023,95,short film (live action),cyrus neshvad,the red suitcase,False,2022The Red Suitcase
10750,2022,2023,95,writing (adapted screenplay),written by rian johnson,glass onion a knives out mystery,False,2022Glass Onion: A Knives Out Mystery


In [9]:
# create a connection to a file called 'file.db'
con = duckdb.connect('imdb_reviews.db')

# create a table and load data into it
con.sql("CREATE TABLE IF NOT EXISTS oscars AS SELECT * FROM oscars")


con.close()